In [1]:
import time
import os
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from kafka import KafkaProducer

In [2]:
#Đẩy lên kafka
producer = KafkaProducer(bootstrap_servers="localhost:9092")

with open("VCB Historical Data.csv", "r") as f:
    for line in f:
        producer.send("n2_ict", line.encode())

producer.flush()

In [ ]:

class FileChangeHandler(FileSystemEventHandler):
    def __init__(self, file_path, kafka_topic, producer):
        self.file_path = file_path
        self.kafka_topic = kafka_topic
        self.producer = producer
        self.last_position = os.path.getsize(file_path)
    
    def on_modified(self, event):
        if event.src_path == self.file_path:
            with open(self.file_path, 'r') as file:
                file.seek(self.last_position)
                new_lines = file.readlines()
                self.last_position = file.tell()
                
                for line in new_lines:
                    self.producer.send(self.kafka_topic, line.encode('utf-8'))
                    print(f"Sent to Kafka: {line.strip()}")

def watch_file(file_path, kafka_topic):
    producer = KafkaProducer(bootstrap_servers='localhost:9092')
    event_handler = FileChangeHandler(file_path, kafka_topic, producer)
    
    # Kiểm tra và in ra đường dẫn
    directory = os.path.dirname(file_path)
    print(f"Watching directory: {directory}")
    
    observer = Observer()
    observer.schedule(event_handler, path=directory, recursive=False)
    observer.start()
    
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

# Sử dụng đường dẫn tuyệt đối
file_path = r"D:\SIC-BD1\Result\VCB Historical Data.csv"
watch_file(file_path, 'n2_ict')


Watching directory: D:\SIC-BD1\Result
Sent to Kafka: 15/12/2024,93000,93000,93200,93000
Sent to Kafka: VCB,15/12/2024,93000,93200,93000,93000
Sent to Kafka: VCB,12/12/2024,93200,93600,92900,93100
Sent to Kafka: VCB,11/12/2024,93500,94400,93500,93500
Sent to Kafka: VCB,10/12/2024,94500,94800,93200,93300
Sent to Kafka: VCB,09/12/2024,95300,95300,94500,94500
Sent to Kafka: VCB,08/12/2024,94200,95300,93900,95300
Sent to Kafka: VCB,05/12/2024,94000,95300,93900,94200
Sent to Kafka: VCB,04/12/2024,93000,94600,93000,94000
Sent to Kafka: VCB,03/12/2024,92800,93900,92500,93500
Sent to Kafka: VCB,02/12/2024,94400,94400,92600,93000
Sent to Kafka: VCB,01/12/2024,94900,95100,93900,94200
Sent to Kafka: VCB,28/11/2024,92600,93300,92500,93300
Sent to Kafka: VCB,27/11/2024,93100,93500,92500,92600
Sent to Kafka: VCB,26/11/2024,92300,93200,92300,92700
Sent to Kafka: VCB,25/11/2024,91400,92900,91400,92400
Sent to Kafka: VCB,24/11/2024,90900,92200,90900,91300
Sent to Kafka: VCB,21/11/2024,91000,91300,90600,